# Project 3

<b> Group members: </b>
1.	Dianela Perdomo 
2.	Hendyel Pacheco
3.	Lihe Liu
4.	Pornpamol Pattamanont

## Data:
The data file for this problem set is here:
```bash
/ufrc/zoo6927/share/Class_Files/data/flights.May2017-Apr2018.csv 
```

## Plan:

1. Two tables we are going to create are:
    * Airports table
    * Flights table with all flights infos 
2. The first table has 5 columns and is the airports table: 
    * id, code, city, state, name. 
    * The primary key in the first table is the ID which is autoincremented..
3. The second table is 'flights table’ which has:
    * fl_data, airline_id, and 19 columns total of flight data. it includes all flights
    * The foreign key is the airports.code from the first table.
4. We use 'join' function to link between these 2 tables.


## Solution:

<b> Create empty database: </b>

In [43]:
from sqlalchemy import create_engine
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey
from sqlalchemy import DateTime, Boolean
from sqlalchemy import exists
from sqlalchemy import sql, select, join, desc

# Create a sqlite database 
engine = create_engine('sqlite:///pj3_flights.sqlite')

metadata = MetaData(engine)

# Try to load Airports info from database, if not there, create it.
try:
    Airports=Table('Airports', metadata, autoload=True)
except:
    Airports = Table ('Airports', metadata,
                Column('ID', Integer, autoincrement=True),
                Column('Code', String, primary_key=True),
                Column('City', String),
                Column('State', String),
                Column('Name', String),
               )

# Same for Flights table.
try:
    Flights=Table('Flights', metadata, autoload=True)
except:
    Flights = Table ('Flights', metadata,
                 Column('Fl_date', DateTime),
                 Column('Airline_ID', String),
                 Column('Origin', String, ForeignKey("Airports.Code")),
                 Column('Destination', String, ForeignKey("Airports.Code")),
                 Column('Dep_Time', String),
                 Column('Dep_Delay_New', Integer),
                 Column('Arr_Time', String),
                 Column('Arr_Delay_New', Integer),
                 Column('Cancelled', Boolean),
                 Column('Cancellation_Code', String),
                 Column('Diverted', Boolean),
                 Column('Air_Time', String),
                 Column('Flights', Integer),
                 Column('Distance', Integer),
                 Column('Carrier_Delay', Integer),
                 Column('Weather_Delay', Integer),
                 Column('NAS_Delay', Integer),
                 Column('Security_Delay', Integer),
                 Column('Late_Aircraft_Delay', Integer)
                )
 

metadata.create_all(engine)


In [44]:
import csv

flights=open("/ufrc/zoo6927/share/Class_Files/data/flights.1K.csv")


reader = csv.DictReader(flights)

Airport_dict = {}

# Read through the file and make a dictionary for airport codes.
# This gets a unique list of airport codes.
for Line in reader:
    if Line['ORIGIN'] not in Airport_dict:
        Airport_dict[Line['ORIGIN']]=[Line['ORIGIN_CITY_NAME'], Line['ORIGIN_STATE_ABR']]

    if Line['DEST'] not in Airport_dict:
        Airport_dict[Line['DEST']]=[Line['DEST_CITY_NAME'], Line['DEST_STATE_ABR']]

<b> Add data into dataframe: </b>

In [45]:
import csv
flights=open("/ufrc/zoo6927/share/Class_Files/data/flights.1K.csv")


reader = csv.DictReader(flights)

Airport_dict={}

# Read through the file and make a dictionary for airport codes.
# This gets a unique list of airport codes.
for Line in reader:
    if Line['ORIGIN'] not in Airport_dict:
        Airport_dict[Line['ORIGIN']]=[Line['ORIGIN_CITY_NAME'], Line['ORIGIN_STATE_ABR']]

    if Line['DEST'] not in Airport_dict:
        Airport_dict[Line['DEST']]=[Line['DEST_CITY_NAME'], Line['DEST_STATE_ABR']]

print(Airport_dict)

{'LAX': ['Los Angeles, CA', 'CA'], 'IAD': ['Washington, DC', 'VA'], 'SAN': ['San Diego, CA', 'CA'], 'SFO': ['San Francisco, CA', 'CA'], 'EWR': ['Newark, NJ', 'NJ'], 'JFK': ['New York, NY', 'NY'], 'OGG': ['Kahului, HI', 'HI'], 'SEA': ['Seattle, WA', 'WA'], 'DCA': ['Washington, DC', 'VA'], 'ORD': ['Chicago, IL', 'IL'], 'AUS': ['Austin, TX', 'TX'], 'PDX': ['Portland, OR', 'OR'], 'LAS': ['Las Vegas, NV', 'NV'], 'MCO': ['Orlando, FL', 'FL'], 'FLL': ['Fort Lauderdale, FL', 'FL'], 'BOS': ['Boston, MA', 'MA'], 'HNL': ['Honolulu, HI', 'HI'], 'PSP': ['Palm Springs, CA', 'CA'], 'DAL': ['Dallas, TX', 'TX'], 'LGA': ['New York, NY', 'NY'], 'DEN': ['Denver, CO', 'CO'], 'SNA': ['Santa Ana, CA', 'CA'], 'IAH': ['Houston, TX', 'TX'], 'MEM': ['Memphis, TN', 'TN'], 'CLE': ['Cleveland, OH', 'OH'], 'AVL': ['Asheville, NC', 'NC'], 'ONT': ['Ontario, CA', 'CA'], 'MSP': ['Minneapolis, MN', 'MN'], 'MIA': ['Miami, FL', 'FL'], 'SAV': ['Savannah, GA', 'GA'], 'ATL': ['Atlanta, GA', 'GA'], 'TPA': ['Tampa, FL', 'FL'], 

In [46]:

# Add the Airport_dict codes to the Airports table

conn = engine.connect()


def insert_airport(code,city,state):
    ins = Airports.insert().values(Code=code,
                                 City=city,
                                 State=state)
    
    result = conn.execute(ins)

for key, value in Airport_dict.items(): 
    insert_airport(key, value[0], value[1])

In [47]:
import datetime
import pandas as pd
import csv

def to_date(dates, lookup=False, **args):
    if lookup:
        return dates.map({v: pd.to_datetime(v, **args) for v in dates.unique()})
    return pd.to_datetime(dates, **args)

conn = engine.connect()

# Close the file
flights.close()

# Re-open to get flight data


flights=open("/ufrc/zoo6927/share/Class_Files/data/flights.1K.csv")

reader = csv.DictReader(flights)

for Line in reader:

    ins=Flights.insert().values(Fl_date=to_date(Line['FL_DATE']),
                                Airline_ID = Line['AIRLINE_ID'],
                                Origin = Line['ORIGIN'],
                                Destination = Line['DEST'],
                                Dep_Time = Line['DEP_TIME'],
                                Dep_Delay_New = Line['DEP_DELAY_NEW'],
                                Arr_Time = Line['ARR_TIME'],
                                Arr_Delay_New = Line['ARR_DELAY_NEW'],
                                Cancelled = int(float((Line['CANCELLED']))),
                                Cancellation_Code = Line['CANCELLATION_CODE'],
                                Diverted = int(float((Line['DIVERTED']))),
                                Air_Time = Line['AIR_TIME'],
                                Flights = Line['FLIGHTS'],
                                Distance = Line['DISTANCE'],
                                Carrier_Delay = Line['CARRIER_DELAY'],
                                Weather_Delay = Line['WEATHER_DELAY'],
                                NAS_Delay = Line['NAS_DELAY'],
                                Security_Delay = Line['SECURITY_DELAY'],
                                Late_Aircraft_Delay = Line['LATE_AIRCRAFT_DELAY']
                                          )
    result = conn.execute(ins)

## Select, Where and Join

**Select all origin and destination pairs, limit = 10**


In [48]:
from sqlalchemy import select

query1 = select([Flights.c.Origin, Flights.c.Destination])\
        .limit(10)
             
result1 = conn.execute(query1)
for row in result1:
    print(row)

('LAX', 'IAD')
('LAX', 'IAD')
('SAN', 'SFO')
('SFO', 'EWR')
('JFK', 'SFO')
('LAX', 'OGG')
('OGG', 'LAX')
('SFO', 'JFK')
('LAX', 'EWR')
('EWR', 'LAX')


**Count the total number of flights delayed more than 15 minutes**

In [49]:
from sqlalchemy import select
from sqlalchemy.sql.expression import func

#how to do count

query2 = select([func.count(Flights.c.Dep_Delay_New)])\
        .where(Flights.c.Dep_Delay_New > 15)
             
result2 = conn.execute(query2)
for row in result2:
    print(row)

(229,)


**Count the total number of flights delayed more than 15 minutes by Origin**

In [51]:
from sqlalchemy import select
from sqlalchemy.sql.expression import func

#Count 

query3 = select([Flights.c.Origin, func.count(Flights.c.Dep_Delay_New)])\
        .where(Flights.c.Dep_Delay_New > 15)\
        .group_by(Flights.c.Origin)
             
result3 = conn.execute(query3)
for row in result3:
    print(row)
    


('ALB', 1)
('ATL', 4)
('AUS', 1)
('AVL', 1)
('BOS', 9)
('BUR', 1)
('CMH', 1)
('DAL', 3)
('DCA', 1)
('DEN', 21)
('DFW', 2)
('EWR', 18)
('FLL', 2)
('HNL', 1)
('IAD', 13)
('IAH', 9)
('JFK', 6)
('LAS', 10)
('LAX', 25)
('LGA', 9)
('MCO', 1)
('MFR', 1)
('MIA', 2)
('MSP', 1)
('MSY', 3)
('OGG', 1)
('ORD', 29)
('PBI', 1)
('PDX', 1)
('PIT', 1)
('RDU', 1)
('ROC', 1)
('SAN', 1)
('SEA', 7)
('SFO', 36)
('SLC', 1)
('SNA', 1)
('TPA', 2)


**Join**


In [52]:
query4 = select([Flights.c.Distance, Flights.c.Air_Time, Airports.c.City, Airports.c.State])\
    .where(Flights.c.Origin==Airports.c.Code)\
    .limit(10)

result4 = conn.execute(query4)
for row in result4:
    print(row)

(2288, '267.00', 'Los Angeles, CA', 'CA')
(2288, '253.00', 'Los Angeles, CA', 'CA')
(447, '68.00', 'San Diego, CA', 'CA')
(2565, '312.00', 'San Francisco, CA', 'CA')
(2586, '331.00', 'New York, NY', 'NY')
(2485, '309.00', 'Los Angeles, CA', 'CA')
(2485, '302.00', 'Kahului, HI', 'HI')
(2586, '288.00', 'San Francisco, CA', 'CA')
(2454, '290.00', 'Los Angeles, CA', 'CA')
(2454, '321.00', 'Newark, NJ', 'NJ')


In [53]:
# Using .join

meta = MetaData(engine, reflect=True)
Flights_t = meta.tables['Flights']
Airports_t = meta.tables['Airports']


join_obj = Flights_t.join(Airports_t,
                         Flights_t.c.Origin==Airports_t.c.Code)

sel_st = select(
   [Flights_t.c.Distance, Flights_t.c.Air_Time, Airports_t.c.City, Airports_t.c.State]).select_from(join_obj)

result5 = conn.execute(sel_st)

for row in result5:
    print(row)

/apps/jupyterhub/0.9.2/lib/python3.6/site-packages/ipykernel_launcher.py:3: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  This is separate from the ipykernel package so we can avoid doing imports until


(2288, '267.00', 'Los Angeles, CA', 'CA')
(2288, '253.00', 'Los Angeles, CA', 'CA')
(447, '68.00', 'San Diego, CA', 'CA')
(2565, '312.00', 'San Francisco, CA', 'CA')
(2586, '331.00', 'New York, NY', 'NY')
(2485, '309.00', 'Los Angeles, CA', 'CA')
(2485, '302.00', 'Kahului, HI', 'HI')
(2586, '288.00', 'San Francisco, CA', 'CA')
(2454, '290.00', 'Los Angeles, CA', 'CA')
(2454, '321.00', 'Newark, NJ', 'NJ')
(2454, '279.00', 'Los Angeles, CA', 'CA')
(2454, '320.00', 'Newark, NJ', 'NJ')
(2454, '291.00', 'Los Angeles, CA', 'CA')
(2454, '326.00', 'Newark, NJ', 'NJ')
(2454, '324.00', 'Newark, NJ', 'NJ')
(2454, '266.00', 'Los Angeles, CA', 'CA')
(679, '98.00', 'San Francisco, CA', 'CA')
(679, '93.00', 'Seattle, WA', 'WA')
(2565, '315.00', 'San Francisco, CA', 'CA')
(2565, '292.00', 'San Francisco, CA', 'CA')
(954, '124.00', 'Seattle, WA', 'WA')
(2565, '278.00', 'San Francisco, CA', 'CA')
(2565, '328.00', 'Newark, NJ', 'NJ')
(2565, '325.00', 'Newark, NJ', 'NJ')
(337, '56.00', 'San Francisco, CA'